# Loading an existing MODFLOW 6 model and adding a GWT transport model with FloPy

<img src="images/watershed_domain.png" width="1000" height="500">

A MODFLOW 6 model will be developed of the domain shown above. This model simulation is based on the structured watershed example in [Hughes, J.D., Langevin, C.D., Paulinski, S.R., Larsen, J.D. and Brakenhoff, D. (2024), FloPy Workflows for Creating Structured and Unstructured MODFLOW Models. Groundwater, 62: 124-139. https://doi.org/10.1111/gwat.13327](https://doi.org/10.1111/gwat.13327).

The model domain is be discretized into 5 layers, 41 rows, and 73 columns. The hydraulic conditivity was modifed to add heterogenity in layers 2 and 4. 

MODFLOW does not require that input data be provided in specific units (for example, SI units) instead it only requires that consistent units be used. As a result all input data should be specified with a length unit of feet and a time unit of days.

<img src="images/watershed_hydraulic_conductivity.png" width="1000" height="250">

In [ ]:
%matplotlib inline
import pathlib as pl

import flopy
import matplotlib as mpl
import matplotlib.transforms as mtransforms
import matplotlib.pyplot as plt
import numpy as np

Before loading the existing MODFLOW 6 groundwater flow model you should define the simulation workspace (`ws`) where the model files are located and the simulation name (`name`). The `ws` should be set to `"simbase"` and `name` should be set to `"watershed"`.

In [ ]:
ws = pl.Path("simbase")
name = "watershed"

## Load the model

Load the existing groundwater flow model

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=ws)

Change the existing simulation workspace to `"class-model"` so we do not alter the existing model files. The simulation workspace can be changed using `sim.set_sim_path(ws)`.

In [ ]:
ws = pl.Path("class-model")
sim.set_sim_path(ws)

Get the groundwater flow model from the simulation using `sim.get_model()`. The name of the gwf_model is `"gwf"`.

In [ ]:
gwf_name = "gwf"
gwf = sim.get_model(gwf_name)

## Add the groundwater transport model

Create the groundwater transport model object (`gwt`) using `flopy.mf6.ModflowGwf()`. Make sure to include the simulation object (`sim`) as the first variable in the groundwater transport model object and set `modelname` to `gwt`. Use `Shift-Tab` to see the optional variables that can be specified.

So that the file names are unique for the groundwater transport model make sure that you

In [ ]:
gwt_name = "gwt"

conc_start = 0.0
porosity = 0.35

In [ ]:
gwt_name = "gwt"
gwt = flopy.mf6.ModflowGwt(
    sim,
    modelname=gwt_name,
    print_input=False,
    save_flows=True,
)


### Create an IMS package for the GWT model

Create the IMS package for the groundwater transport model using `flopy.mf6.ModflowIms()` and register it for use with the groundwater transport model using `sim.register_ims_package()`.

Make sure to define the `filename=` for the IMS package to `gwt.ims`.

In [ ]:
imsgwt = flopy.mf6.ModflowIms(
        sim,
        complexity="complex",
        print_option="SUMMARY",
        linear_acceleration="bicgstab",
        outer_maximum=1000,
        inner_maximum=100,
        outer_dvclose=1e-4,
        inner_dvclose=1e-5,
        filename=f"{gwt_name}.ims",
    )

sim.register_ims_package(imsgwt, [gwt_name])

### Create the discretization package for the GWT model

Create the discretization package using `flopy.mf6.ModflowGwtdis()`. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `DIS` package (`flopy.mf6.ModflowGwtdis()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwt-dis.html).

We will extract the discretization data from the groundwater flow model.

In [ ]:
pak = gwf.dis
nlay, nrow, ncol = pak.nlay.array, pak.nrow.array, pak.ncol.array
shape3d = (nlay, nrow, ncol)
xorigin, yorigin = pak.xorigin.array, pak.yorigin.array
delr, delc = pak.delr.array, pak.delc.array
top, botm = pak.top.array, pak.botm.array
idomain = pak.idomain.array

In [ ]:
dis = flopy.mf6.ModflowGwtdis(
    gwt,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    idomain=idomain,
    top=top,
    botm=botm,
    xorigin=xorigin,
    yorigin=yorigin,
    # filename=f"{gwt_name}.dis",
)

### Create the initial conditions for the GWT model

Create the initial conditions package for the groundwater tansport model using `flopy.mf6.ModflowGwtic()` and set the initial concentration (`strt`) to 0.0.

In [ ]:
# initial conditions
ic = flopy.mf6.ModflowGwtic(
    gwt, 
    strt=0.0, 
    # filename=f"{gwt_name}.ic",
    )



In [ ]:
# advection
adv = flopy.mf6.ModflowGwtadv(
    gwt, 
    scheme="upstream", 
    # filename=f"{gwt_name}.adv",
    )



In [ ]:
# mobile storage and transfer
mst = flopy.mf6.ModflowGwtmst(
    gwt, 
    porosity=0.35,
    zero_order_decay=True,
    decay=-1.0 / 365.25,
    # filename=f"{gwt_name}.mst"
)



In [ ]:
# sources and mixing
sourcerecarray = [
    ("rch_original", "AUX", "CONCENTRATION"),
]
ssm = flopy.mf6.ModflowGwtssm(
    gwt, 
    # sources=sourcerecarray, 
    # filename=f"{gwt_name}.ssm",
)



### Build output control

Define the output control package (`OC`) for the model using the `flopy.mf6.ModflowGwtoc()` method to `[('CONCENTRATION', 'ALL'), ('BUDGET', 'ALL')]` to save the head and flow for the model. Also the head (`concentration_filerecord`) and cell-by-cell flow (`budget_filerecord`) files should be set to `f"{gwt_name}.ucn"` and `f"{gwt_name}.cbc"`, respectively. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `OC` package (`flopy.mf6.ModflowGwtoc()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwt-oc.html).

In [ ]:
# output control
oc = flopy.mf6.ModflowGwtoc(
        gwt,
        budget_filerecord=f"{gwt_name}.cbc",
        concentration_filerecord=f"{gwt_name}.ucn",
        saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")],
        # filename=f"{gwt_name}.oc",
    )

`pmv = flopy.plot.PlotMapView(model=gwf)` and `pxs = flopy.plot.PlotCrossSection(model=gwf, line={"row": 20})` can be used to confirm that the discretization is correctly defined.

In [ ]:
gwfgwt = flopy.mf6.ModflowGwfgwt(
    sim,
    exgtype="GWF6-GWT6",
    exgmnamea=gwf_name,
    exgmnameb=gwt_name,
    filename="gwfgwt.exg",
)

### Write the model files and run the model

Write the MODFLOW 6 model files using `sim.write_simulation()`. Use `Shift-Tab` to see the optional variables that can be specified for `.write_simulation()`.

In [ ]:
sim.write_simulation()

Run the model using `sim.run_simulation()`, which will run the MODFLOW 6 executable installed in the Miniforge class environment (`pyclass`) and the MODFLOW 6 model files created with `.write_simulation()`. Use `Shift-Tab` to see the optional variables that can be specified for `.run_simulation()`.

In [ ]:
sim.run_simulation()

## Post-process the results

Load the concentrations and specific discharge from the ucn and cbc files. The concentration file can be loaded with the `gwt.output.concentration().get_data()` method. The cell-by-cell file can be loaded with the `gwf.output.budget().get_data()` method. 

First, we will get the simulation times available in the ucn file using `gwt.output.concentration().get_times()`.


In [ ]:
cobj = gwt.output.concentration()

In [ ]:
times = cobj.get_times()

In [ ]:
totim = times[-1]
vmin, vmax = 0.0, totim / 365.25

In [ ]:
times[0]

Retrieve the `'DATA-SPDIS'` data type from the cell-by-cell file. Name the specific discharge data `spd`.

Cell-by-cell data is returned as a list so access the data by using `spd = gwf.output.budget().get_data(text="DATA-SPDIS")[0]`.

In [ ]:
budobj = gwf.output.budget()

In [ ]:
spd = gwf.output.budget().get_data(totim=totim, text="DATA-SPDIS")[0]
qx, qy, qz = flopy.utils.get_specific_discharge(spd, model=gwt)

In [ ]:
spd["qx"]

In [ ]:
conc = gwt.output.concentration().get_data(totim=totim)

In [ ]:
def plot_intro_model(model, cobj, budobj, totim=3652.5, vmin=None, vmax=None, layer=0):
    mosaic = [
        ["a", "a", "a", "c"],
        ["a", "a", "a", "c"],
        ["a", "a", "a", "c"],
        ["b", "b", "b", "."],
        ]
    
    c = cobj.get_data(totim=totim)
    spd = budobj.get_data(totim=totim, text="DATA-SPDIS")[0]
    qx, qy, qz = flopy.utils.get_specific_discharge(spd, model=gwt)

    column = 25
    row = 24
    b_y = gwt.modelgrid.ycellcenters[row, 0]
    c_x = gwt.modelgrid.xcellcenters[0, column]
    
    with flopy.plot.styles.USGSMap():
        fig, axd = plt.subplot_mosaic(
            mosaic,
            figsize=(10,8),
            layout="constrained",
            )
        
        ax = axd["a"]
        mv = flopy.plot.PlotMapView(model=model, ax=ax, layer=layer)
        pa = mv.plot_array(c, vmin=vmin, vmax=vmax)
        mv.plot_grid(lw=0.5, color="black")
        mv.plot_vector(qx, qy)
        x0, x1 = ax.get_xlim()
        y0, y1 = ax.get_ylim()
        ax.axhline(b_y, lw=2.0, color="red", ls=":")
        ax.axvline(c_x, lw=2.0, color="red", ls=":")
        ax.set_xticklabels([])
        ax.set_ylabel("y-position, m")

        # cb = plt.colorbar(pa, shrink=0.5, orientation="horizontal")        

        ax = axd["b"]
        xs = flopy.plot.PlotCrossSection(model=model, ax=ax, line={"row": row})
        pa = xs.plot_array(conc, vmin=vmin, vmax=vmax)
        xs.plot_grid(lw=0.5, color="black")
        ax.axvline(c_x, lw=2.0, color="red", ls=":")
        # xs.plot_vector(qx, qy, qz)
        # cb = plt.colorbar(pa, shrink=0.5, orientation="horizontal")
        ax.set_ylabel("Elevation, m")
        ax.set_xlabel("x-position, m")


        ax = axd["c"]
        xs = flopy.plot.PlotCrossSection(
            model=model, 
            ax=ax, 
            geographic_coords=False, 
            line={"column": column},
            )
        pa = xs.plot_array(conc, vmin=vmin, vmax=vmax)
        pg = xs.plot_grid(lw=0.5, color="black")

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        xc, yc = 0.5 * sum(xlim), 0.5 * sum(ylim)
        rotation_point = np.array([xc, yc])
        angle_deg = -90

        transform = (
            mtransforms.Affine2D().translate(*-rotation_point) +  # Translate to origin
            mtransforms.Affine2D().rotate_deg(angle_deg) +        # Rotate
            mtransforms.Affine2D().translate(*np.flip(rotation_point))      # Translate back
        )

        pa.set_transform(transform + ax.transData)
        pg.set_transform(transform + ax.transData)

        ax.set_xlim(ylim[::-1])
        ax.set_ylim(xlim)
        ax.axhline(b_y, lw=2.0, color="red", ls=":")
        ax.set_yticklabels([])
        ax.set_xlabel("Elevation, m")


    return fig, axd
        

In [ ]:
plot_intro_model(gwt, cobj, budobj, totim=totim)

In [ ]:
xs_elev = gwt.modelgrid.top_botm[:, :, 25]
ylim = (float(np.floor(xs_elev[-1, :].min())), float(np.ceil(xs_elev[0, :].max())))
ylim

In [ ]:
gwt.modelgrid.xcellcenters[0, ]

In [ ]:
xs = flopy.plot.PlotCrossSection(model=gwt, line={"column": 25})
ax = xs.ax
# ax.set_ylim(ylim)

pa = xs.plot_array(conc, vmin=vmin, vmax=vmax)
pg = xs.plot_grid(lw=0.5, color="black")

xlim = ax.get_xlim()
ylim = ax.get_ylim()
xc, yc = 0.5 * sum(xlim), 0.5 * sum(ylim)
rotation_point = np.array([xc, yc])
print(rotation_point)
print(-rotation_point)

angle_deg = -90
angle_rad = np.deg2rad(angle_deg)

transform = (
    mtransforms.Affine2D().translate(*-rotation_point) +  # Translate to origin
    mtransforms.Affine2D().rotate_deg(angle_deg) +        # Rotate
    mtransforms.Affine2D().translate(*np.flip(rotation_point))      # Translate back
)
pa.set_transform(transform + ax.transData)
pg.set_transform(transform + ax.transData)

# ax.autoscale()

ax.set_xlim(ylim[::-1])
ax.set_ylim(xlim)

# plt.plot(yc, xc, marker="o", color="red", ms=10)



In [ ]:
for col in ax.collections:
    print(col)

In [ ]:
pa.set_offset_transform()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import matplotlib.patches as patches
import numpy as np
import math

fig, ax = plt.subplots()
ax.set_xlim(-5, 10)
ax.set_ylim(-5, 10)
ax.grid(True)

# Define a rectangle
rect = patches.Polygon(np.array([[1, 1], [1, 3], [10, 3], [10, 1]]),
                       fc='blue', alpha=0.5)
ax.add_patch(rect)


# Define the rotation point and angle
rotation_point = np.array([0.5 * (1 + 10), 0.5 * (1 + 3)])
angle_deg = 90
angle_rad = np.deg2rad(angle_deg)

# Create the transformation by chaining operations
transform = (
    mtransforms.Affine2D().translate(*-rotation_point) #+   # Translate to origin
    # mtransforms.Affine2D().rotate_deg(angle_deg) +        # Rotate
    # mtransforms.Affine2D().rotate(angle_rad) +             # Rotate
    # mtransforms.Affine2D().translate(*np.flip(rotation_point))      # Translate back
)

# Apply the transformation
rect.set_transform(transform + ax.transData)

plt.show()


In [ ]:
ax.patches